# AIMO3 Training on OpenMathReasoning Dataset

Fine-tune a math-capable model on the OpenMathReasoning dataset for AIMO3 competition.

## Setup

In [ ]:
# Clone the repo
!git clone https://github.com/kkhskh/1.git
%cd 1

# Install dependencies
!pip install -q datasets transformers peft accelerate torch

In [ ]:
# Authenticate with Hugging Face (for model access)
from huggingface_hub import login

# You'll need to get a token from https://huggingface.co/settings/tokens
# Make sure to accept the terms for any gated models
login()  # This will prompt for your token

In [ ]:
# FIXED: Use smaller model + 4-bit quantization for Colab T4
# model_name = "Qwen/Qwen2.5-Math-7B"  # Too big for T4
model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # SMALLER: ~3GB VRAM with 4-bit
# model_name = "microsoft/Phi-3-mini-4k-instruct"  # Alternative: ~2GB VRAM

# Training configuration - FULL DATASET with Colab Pro
num_train_epochs = 3
max_samples = 0  # 0 = use FULL dataset (~1.8M samples)
batch_size = 4  # Higher batch size for A100/V100
learning_rate = 1e-5  # Lower learning rate for full training

In [ ]:
# Run the training
import os

# Set environment variables
os.environ['MODEL_NAME'] = model_name
os.environ['NUM_EPOCHS'] = str(num_train_epochs)
os.environ['MAX_SAMPLES'] = str(max_samples)
os.environ['MAX_COT'] = str(0)
os.environ['MAX_TIR'] = str(0)
os.environ['MAX_STEPS'] = str(600)
os.environ['BATCH_SIZE'] = str(batch_size)
os.environ['LEARNING_RATE'] = str(learning_rate)

# Run training
!python train_openmath.py

## After Training

1. Download the trained model from `./models/openmath-finetuned/`
2. Test it locally or use it in your competition pipeline
3. For inference, you can use the model with your existing pipeline

In [ ]:
# Test the trained model (after training)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the trained model
model_path = "./models/openmath-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Test problem
test_problem = "Solve for x: 2x + 3 = 7"
prompt = f"Problem: {test_problem}\n\nSolution: "

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.1,
    do_sample=True
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Model response:")
print(response)